Import Python modules...

In [ ]:
from __future__ import print_function

import os
import sys
import time
import re

import requests

print(time.asctime())

**The URL PATH**

The MW REST URL consists of three main parts, separated by forward slashes, after the common prefix specifying the invariant base URL (https://www.metabolomicsworkbench.org/rest/):

https://www.metabolomicsworkbench.org/rest/context/input_specification/output_specification
    
Part 1: The context determines the type of data to be accessed from the Metabolomics Workbench, such as metadata or results related to the submitted studies, data from metabolites, genes/proteins and analytical chemistry databases as well as other services related to mass spectrometry and metabolite identification:

context = study | compound | refmet | gene | protein | moverz | exactmass

Part 2: The input specification consists of two required parameters describing the REST request:

input_specification = input_item/input_value

Part 3: The output specification consists of two parameters describing the output generated by the REST request:

output_specification = output_item/(output_format)

The first parameter is required in most cases. The second parameter is optional. The input and output specifications are context sensitive. The context determines the values allowed for the remaining parameters in the input and output specifications as detailed in the sections below.


Setup MW REST base URL...

In [ ]:
MWBaseURL = "https://www.metabolomicsworkbench.org/rest"

**The “study” context**

The "study" context refers to the studies available in the Metabolomics Workbench (www.metabolomicsworkbench.org), a public repository for metabolomics metadata and experimental data spanning various species and experimental platforms, metabolite standards, metabolite structures, protocols, tutorials and training material, and other educational resources. It provides a computational platform to integrate, analyze, track, deposit, and disseminate large volumes of heterogeneous data from a wide variety of metabolomics studies including Mass Spectrometry (MS) and Nuclear Magnetic Resonance (NMR) spectrometry data spanning a variety of species covering all the major taxonomic categories including humans and other mammals, plants, insects, invertebrates, and microorganisms. This context provides access to a variety of data associated with studies such as study summary, experimental factors for study design, analysis information, metabolites and results data, sample source and species etc.

context = study

input_item = study_id | study_title | institute | last_name | analysis_id | metabolite_id

input_value = input_item_value

output_item = summary | factors | analysis | metabolites | mwtab | source | species | disease | number_of_metabolites | data | datatable | untarg_studies | untarg_factors | untarg_data

output_format = txt | json (Default: json)



**Retrieve and process study data for in JSON format**


Setup REST URL to retrieve data for a study ID...

In [ ]:
MWDataURL = MWBaseURL + "/study/study_id/ST000001/summary/"

Execute REST request using "request" module...

In [ ]:
print("Initiating request: %s" % MWDataURL)
    
Response = requests.get(MWDataURL)

Check "request" status...

In [ ]:
print("\nStatus Code: %d" % (Response.status_code))

if Response.status_code != 200:
    print("Request failed: status_code: %d" % Response.status_code)

Process JSON results...

In [ ]:
print("\nAvailable data for a study summary:\n")

Results = Response.json()

for ResultType in Results:
    ResultValue = Results[ResultType]
    print("%s: %s" % (ResultType, ResultValue))


**Retrieve and process study data for multiple studies in JSON format**


A substring may be specified as a study ID to retrieve data for multiple studies. The specified study ID substring is matched against all study IDs to retrieve data for matched studies. The study ID substring "ST" matches against all available studies.

Setup REST URL to retrieve data for studies ST000010 to ST000019 by using ST00001 as study ID...

In [ ]:
MWDataURL = MWBaseURL + "/study/study_id/ST00001/summary/"

Execute REST request and check the status...

In [ ]:
print("Initiating request: %s" % MWDataURL)
    
Response = requests.get(MWDataURL)

print("\nStatus Code: %d" % (Response.status_code))

if Response.status_code != 200:
    print("Request failed: status_code: %d" % Response.status_code)

Process JSON results...

In [ ]:
print("\nAvailable data for studies:\n")

Results = Response.json()

StudiesCount = 0
for ResultNum in Results:
    StudiesCount += 1
    print("\nResultNum: %s\n" % ResultNum)
    
    for ResultType in Results[ResultNum]:
        ResultValue = Results[ResultNum][ResultType]    
        print("%s: %s" % (ResultType, ResultValue))

print("\nTotal number of studies matched: %d" % StudiesCount)
    


**Retrieve and process study data in text format**


Set up REST URL...

In [ ]:
MWDataURL = MWBaseURL + "/study/study_id/ST000001/summary/txt"

Execute REST request using "request" module...

In [ ]:
print("Initiating request: %s" % MWDataURL)
    
Response = requests.get(MWDataURL)

Check "request" status...

In [ ]:
print("\nStatus Code: %d" % (Response.status_code))

if Response.status_code != 200:
    print("Request failed: status_code: %d" % Response.status_code)

Process text results...

In [ ]:
print("\nAvailable data for study summary:\n")

Results = Response.text
for Result in Results.split("\n"):
    Words = Result.split("\t")
    if len(Words) != 2:
        continue
    
    ResultType, ResultValue = Result.split("\t")
    print("%s: %s" % (ResultType, ResultValue))


**Retrieve and process data for multiple studies in text format**


Setup REST URL to retrieve data for studies ST000010 to ST000019 by using ST00001 as study ID...

In [ ]:
MWDataURL = MWBaseURL + "/study/study_id/ST00001/summary/txt"

Execute REST request and check the status...

In [ ]:
print("Initiating request: %s" % MWDataURL)
    
Response = requests.get(MWDataURL)

print("\nStatus Code: %d" % (Response.status_code))

if Response.status_code != 200:
    print("Request failed: status_code: %d" % Response.status_code)

Process text results...

In [ ]:
print("\nAvailable summary data for studies:\n")

Results = Response.text

StudiesCount = 0
for Result in Results.split("\n"):
    Words = Result.split("\t")
    if len(Words) != 2:
        print("")
        continue
    
    ResultType, ResultValue = Words
    if re.match("^study_id$", ResultType, re.I):
        StudiesCount += 1
    print("%s: %s" % (ResultType, ResultValue))

print("\nTotal number of studies matched: %d" % StudiesCount)